import spacy# Preprocess Discharge Summaries

In [1]:
import os 
myPath = os.path.abspath(os.path.join("../data/mimic_dict.txt", os.pardir))
print(myPath)

/Users/rajvardhan_deshmukh/Documents/GitHub/SyntheticMedicalDataGeneration/data


In [2]:
# !conda install -c conda-forge spacy -y
# !conda install -c conda-forge pyspellchecker -y
# !conda install -c conda-forge flashtext -y

In [3]:
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_md-0.5.1.tar.gz

In [23]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import pandas as pd
import numpy as np
import os
import psycopg2
import sqlalchemy
import string
import spacy
from spacy.symbols import ORTH
import scispacy
from collections import Counter
import re
from datetime import date, datetime, timedelta
import random
from sklearn.model_selection import GroupShuffleSplit, StratifiedShuffleSplit
from spellchecker import SpellChecker
from flashtext import KeywordProcessor
import nltk
from dateutil import parser
from bs4 import BeautifulSoup

In [40]:
#DEMO data
# cnx = sqlalchemy.create_engine('postgresql+psycopg2://btvdksxhyqtmjs:74db647f02a6f661c9a9ef888e24cebe9b17cfd2cc7566c51556096ec9977964\
# @ec2-54-211-255-161.compute-1.amazonaws.com/dc8973qq7atsb1', connect_args={'options': '-csearch_path={}'.format(dbschema)})

#COMPLETE data
dbschema='public'
cnx = sqlalchemy.create_engine('postgresql+psycopg2://njolzisalnylqx:31b9211af0689fdc75c296546539fd3d86b8c8135d277924a0659bcfe735adb3\
@ec2-3-219-135-162.compute-1.amazonaws.com/d6di1ovvoadbog', connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [21]:
patients_table = pd.read_sql_query('''
            SELECT DISTINCT "SUBJECT_ID", "DOB", "GENDER"
            FROM "PATIENTS"
            ''',cnx)

ne_table = pd.read_sql_query('''
            SELECT "SUBJECT_ID", "HADM_ID", "CATEGORY", "CHARTDATE", "ROW_ID", "TEXT"
            FROM "NOTEEVENTS" n
            WHERE n."CATEGORY" = 'Discharge summary'
            -- GROUP BY "SUBJECT_ID", "HADM_ID"
            ''',cnx)


patients_ne = pd.merge(patients_table,ne_table, on=["SUBJECT_ID"])

In [33]:
patients_ne["CHARTDATE"] =  pd.to_datetime(patients_ne["CHARTDATE"].apply(lambda x: parser.parse(x))).dt.date
patients_ne["DOB"] = pd.to_datetime(patients_ne["DOB"].apply(lambda x: parser.parse(x))).dt.date

patients_ne['AGE_AT_NOTEEVENT'] = patients_ne.apply(lambda e: (e['CHARTDATE'] - e['DOB']).days//365, 
                                                    axis=1)

patients_ne = patients_ne[patients_ne['AGE_AT_NOTEEVENT'] >= 18]

patients_ne = patients_ne.sort_values(by=["SUBJECT_ID", "CHARTDATE"],ascending=False)
patients_ne

,SUBJECT_ID,DOB,GENDER,HADM_ID,CATEGORY,CHARTDATE,ROW_ID,TEXT,AGE_AT_NOTEEVENT
12221,99999,2054-01-06,F,113369.0,Discharge summary,2118-01-04,52180,Admission Date: [**2117-12-30**] ...,64
13331,99995,2058-05-29,F,137810.0,Discharge summary,2147-02-11,42710,Admission Date: [**2147-2-8**] D...,88
16398,99992,2078-10-17,F,197084.0,Discharge summary,2144-07-28,41993,Admission Date: [**2144-7-25**] ...,65
12405,99991,2137-04-07,M,151118.0,Discharge summary,2185-01-05,9682,Admission Date: [**2184-12-24**] ...,47
9281,99985,2127-04-08,M,176670.0,Discharge summary,2181-02-12,51770,Admission Date: [**2181-1-27**] ...,53
...,...,...,...,...,...,...,...,...,...
18657,9,2108-01-26,M,150750.0,Discharge summary,2149-11-14,57115,"Name: [**Known lastname 10050**], [**Known fi...",41
18656,9,2108-01-26,M,150750.0,Discharge summary,2149-11-13,20070,Admission Date: [**2149-11-9**] Dischar...,41
27386,6,2109-06-21,F,107064.0,Discharge summary,2175-06-15,20825,Admission Date: [**2175-5-30**] Dischar...,66
41887,4,2143-05-12,F,185777.0,Discharge summary,2191-03-23,4788,Admission Date: [**2191-3-16**] Discharge...,47


Once this data is extracted from the database with the corresponding queries, we begin our preprocessing after referring to Wang(2020):
- We see that text has a lot of special character symbols such as ampersands, equal to, tilda, etc. We remove these along with the stopwords but keep the slashes in the text as if we remove these, it will render the dates useless.
- Convert the dates in the text to a format which is recognised by the scispaCy.
- item Since the texts are typed by humans, there are quite a few spelling errors. These need to be corrected but since our domain is very specific we also need to focus on the out of vocabulary words. Hu(2020) shows that BERT performs well for such a task which is already implemented in contextual spell check by scispaCy.

In [10]:
patients_ne.shape

(55404, 9)

# Preprocessing

In [11]:
# date_regex = re.compile(r'([0-9])-([0-9][0-9]?)-([0-9])') # change date format so spacy can recognise
# newline_regex = re.compile(r'(\\n){3,}') # cap number of consecutive newline characters to 2
# newline_regex2 = re.compile(r'(\\r){3,}') # cap number of consecutive newline characters to 2
# ellipsis_regex = re.compile(r'(\.){2,}')
# tilda_mult_regex = re.compile(r'(~){2,}')
# atsign_mult_regex = re.compile(r'(@){2,}')

# bracket_regex = re.compile(r'(.)(\()(.)')
# bracket_regex2 = re.compile(r'(.)(\))(.)')
# slash_regex = re.compile(r'(.)(\/)([^0-9])')
# slash_regex2 = re.compile(r'([^0-9])(\/)(.)')
# equals_regex = re.compile(r'(.)(=)(.)')
# colon_regex = re.compile(r'(.)(:)(.)')
# sq_bracket_regex = re.compile(r'(.)(\[)(.)')
# dash_regex = re.compile(r'(.)(-)(.)')
# dash_regex2 = re.compile(r'(-)([\S])')
# plus_regex = re.compile(r'(.)(\+)(.)')
# amp_regex = re.compile(r'(.)(&)(.)')
# star_regex = re.compile(r'(.)(\*)(.)') 
# comma_regex = re.compile(r'(.)(,)(.)')
# tilda_regex = re.compile(r'(.)(~)(.)')
# pipe_regex = re.compile(r'(.)(\|)(.)')
# atsign_regex = re.compile(r'(.)(@)(.)')
# dot_regex = re.compile(r'([^.][^0-9])(\.)([^0-9,][^.])')

# dot_regex2 = re.compile(r'([^0-9])(\.)(.)')
# semicol_regex = re.compile(r'(.);(.)')
# caret_regex = re.compile(r'(.)\^(.)')

In [34]:
nlp = spacy.load('en_core_sci_md') # sciSpaCy

nlp.tokenizer.add_special_case(u'<PAR>', [{ORTH: u'<PAR>'}])
nlp.tokenizer.add_special_case(u'<UNK>', [{ORTH: u'<UNK>'}])


word_freq = defaultdict(int)

i = 0

def tokenise_text(text):
    global i
    
    text = str(text).lower()#convert to lower case
#     text = date_regex.sub(r'\1/\2/\3',text)
#     text = newline_regex.sub(r' \\n\\n ',text)
#     text = newline_regex2.sub(r' \\n\\n ',text)
#     text = ellipsis_regex.sub(r'.',text)
#     text = tilda_mult_regex.sub(r'~',text)
#     text = atsign_mult_regex.sub(r'@',text)
    
    text = BeautifulSoup(text).get_text() #remove HTML tags
    text = re.sub('^http[s]?://?\S+|^www.?\S+', '',str(text)) #remove URl    
    text = re.sub(r'([0-9])-([0-9][0-9]?)-([0-9])',r'\1/\2/\3',text) #change yyyy-mm-dd to yyyy/mm/dd
    text = re.sub(r'(\W)(?=\1)', '', text)
    text = re.sub('[^a-zA-Z0-9\/\n@.:~\s\[\]]', '',text)
    
    
    tokens = nlp.tokenizer(text)
    tokenised_text = ""
    
    for token in tokens:
        tokenised_text = tokenised_text + token.text + " "
        word_freq[token.text] += 1
    tokenised_text = tokenised_text.replace("\n"," <PAR> ")
    
    tokenised_text = ' '.join(tokenised_text.split())
    
    if i==0:
        print(i)
    
    i += 1
    if (i % 100) == 0:
        print (i)
    
    return tokenised_text

In [ ]:
patients_ne["TEXT"] = patients_ne["TEXT"].apply(lambda x: tokenise_text(x))

In [38]:
patients_ne.head(5)

,SUBJECT_ID,DOB,GENDER,HADM_ID,CATEGORY,CHARTDATE,ROW_ID,TEXT,AGE_AT_NOTEEVENT
12221,99999,2054-01-06,F,113369.0,Discharge summary,2118-01-04,52180,Admission Date: [**2117-12-30**] ...,64
13331,99995,2058-05-29,F,137810.0,Discharge summary,2147-02-11,42710,Admission Date: [**2147-2-8**] D...,88
16398,99992,2078-10-17,F,197084.0,Discharge summary,2144-07-28,41993,Admission Date: [**2144-7-25**] ...,65
12405,99991,2137-04-07,M,151118.0,Discharge summary,2185-01-05,9682,Admission Date: [**2184-12-24**] ...,47
9281,99985,2127-04-08,M,176670.0,Discharge summary,2181-02-12,51770,Admission Date: [**2181-1-27**] ...,53


Below we isolate the tokens which appear 3 times or fewer. They are mostly misspellings.

In [14]:
infreq_words = [word for word in word_freq.keys() if word_freq[word] <= 3 and word[0].isdigit() == False]
print(len(infreq_words))
sorted(infreq_words)[10000:10050]

95673


['blacktop',
 'blacouts',
 'bladdder',
 'bladde',
 'bladded',
 'bladderincontinence',
 'bladders',
 'bladderscan',
 'blah',
 'blaiteral',
 'blakemoor',
 'blakmore',
 'blanace',
 'blance',
 'blanch',
 'blancheable',
 'blanchible',
 'blancket',
 'blankes',
 'blanketing',
 'blanking',
 'blaoon',
 'blaoting',
 'blare',
 'blase',
 'blask',
 'blastemia',
 'blasting',
 'blastocystisi',
 'blastocystitis',
 'blastocytes',
 'blastoid',
 'blastoma',
 'blastomycoes',
 'blat',
 'blatant',
 'blatent',
 'blaterl',
 'blation',
 'blayton',
 'blbocavernous',
 'blc2',
 'blcoakde',
 'blcok',
 'blcokade',
 'blcoker',
 'bldder',
 'bldk',
 'bldlg',
 'bldneg']

We try and see if we can correct the misspellings using the `pyspellchecker` library by using the Levenshtein Distance algorithm and comparing against a dictionary. We first add the words with >3 occurrence to our dictionary. This is because they include a lot of scientific/medical terms which might not already be there

In [15]:
# freq_words = [word for word in word_freq.keys() if word_freq[word] > 3]
# add_to_dictionary = " ".join(freq_words)
# myPath = os.path.abspath(os.path.join("../data/mimic_dict.txt", os.pardir))
# print(myPath)
# f=open(myPath, "w+")
# f.write(add_to_dictionary)
# f.close()
freq_words = [word for word in word_freq.keys() if word_freq[word] > 3]
add_to_dictionary = " ".join(freq_words)
f=open("../data/mimic_dict.txt", "w+")
f.write(add_to_dictionary)
f.close()

In [16]:
spell = SpellChecker()
spell.distance = 1  # set the distance parameter to just 1 edit away - much quicker
spell.word_frequency.load_text_file('../data/mimic_dict.txt')

In [17]:
misspelled = spell.unknown(infreq_words)
misspell_dict = {}
for i, word in enumerate(misspelled):
    if (word != spell.correction(word)):
        misspell_dict[word] = spell.correction(word)
    if (i % 100 == 0):
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [18]:
print(len(misspell_dict))
list(misspell_dict.items())[:10]

88094


[('nonexpansile', None),
 ('pancr', 'panc'),
 ('negtative', 'negative'),
 ('nonstanding', None),
 ('onlyon', 'onlyone'),
 ('miscategorized', None),
 ('connectiv', 'connective'),
 ('rescussitated', 'recussitated'),
 ('gabentin', 'gabpentin'),
 ('hypocomplimentenemia', None)]

We now have correct spellings for many words in our dictionary that occurred <= 3 times. Anything else will be marked as `UNK`. We will save these as text files to avoid having to run this computation again.

In [19]:
unk_words = [word for word in infreq_words if word not in list(misspell_dict.keys())]
print(len(unk_words))
unk_words[:100]

7579


['ego',
 'foi',
 'facile',
 'skidding',
 'wayside',
 'jetted',
 'crave',
 'capitol',
 'visitations',
 'grouping',
 'drafted',
 'titled',
 'dri',
 'sardine',
 'joins',
 'intensifies',
 'cleaving',
 'membership',
 'sleepwalking',
 'psychoanalyst',
 'explanatory',
 'deactivating',
 'forecasting',
 'sergeant',
 'benches',
 'spore',
 'slits',
 'honors',
 'earmarks',
 'overland',
 'crated',
 'artistic',
 'chore',
 'perpetually',
 'hadthe',
 'diversity',
 'biographical',
 'pocketed',
 'blurs',
 'hydrazine',
 'yoke',
 "din't",
 'commentary',
 'subsections',
 'slamming',
 'prosecutor',
 'loudness',
 'emember',
 'contradicted',
 'secretive',
 'curing',
 'sector',
 'ascended',
 'upl',
 'visas',
 'pies',
 'geneticist',
 'hangover',
 'swears',
 'adenoidal',
 'midmorning',
 'hava',
 'poverty',
 'blase',
 'jutting',
 'stake',
 'categories',
 'heatstroke',
 'shutdown',
 'swamp',
 'greeter',
 'sitdown',
 'unanticipated',
 'possessed',
 'wrenched',
 'comprehensively',
 'meddle',
 'disproved',
 'scooper'

In [20]:
if 'treatement' in misspell_dict.keys():
    print (misspell_dict['treatement'])

In [21]:
np.savetxt('../data/discharge_unk_words.txt', unk_words, fmt='%s', newline=os.linesep)

f=open("../data/discharge_typos.txt", "w+")
for key in misspell_dict:
    if misspell_dict[key] != None:
        f.write(key + '\t' + misspell_dict[key] + '\n')
f.close()

In [22]:
# LOAD TYPOS AND UNKS FROM TXT FILE IF ALREADY GENERATED

#discharge_typos = pd.read_csv('../data/disharge_typos.txt', sep='\t',header=None)
#misspell_dict = {row[0]: row[1] for row in discharge_typos.values}
#unk_words = pd.read_csv('../data/discharge_unk_words.txt', sep='\t', header=None)
#unk_words = list(unk_words[0])

 We will correct spelling mistakes whilst any word left uncorrected will be replaced with `<UNK>`. Instead of using regex, we instead use `FlashText` which is considerably faster for replacing words from a large dictionary.

In [23]:
keywords = list(misspell_dict.keys())
clean = list(misspell_dict.values())

processor = KeywordProcessor()

for keyword_name, clean_name in zip(keywords, clean):
    processor.add_keyword(keyword_name, clean_name)
    
for unk in unk_words:
    processor.add_keyword(unk, "<UNK>")

counter = 0
def fix_typos(text):
    global counter
    
    found = processor.replace_keywords(text)
    
    counter+=1
    if (counter % 100) == 0:
        print (counter)
    
    return found


# apply tokenising function elementwise
df["TEXT"] = df["TEXT"].apply(fix_typos)
df.head()

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

,SUBJECT_ID,DOB,GENDER,HADM_ID,CATEGORY,CHARTDATE,ROW_ID,AGE_AT_NOTEEVENT,TEXT
0,3,2025-04-11 00:00:00,M,145834.0,Discharge summary,2101-10-31,44005,77,admission date : [ 2101/10/20 ] discharge date...
1,4,2143-05-12 00:00:00,F,185777.0,Discharge summary,2191-03-23,4788,48,admission date : [ 2191/3/16 ] discharge date ...
2,6,2109-06-21 00:00:00,F,107064.0,Discharge summary,2175-06-15,20825,66,admission date : [ 2175/5/30 ] discharge date ...
3,9,2108-01-26 00:00:00,M,150750.0,Discharge summary,2149-11-14,57115,42,"name : [ known lastname 10050 ] , [ known firs..."
4,9,2108-01-26 00:00:00,M,150750.0,Discharge summary,2149-11-13,20070,42,admission date : [ 2149/11/9 ] discharge date ...


In [24]:
' '.join(df.head(20)['TEXT'])

'admission date : [ 2101/10/20 ] discharge date : [ 2101/10/31 ] <PAR> <PAR> date of birth : [ 2025/4/11 ] sex : m <PAR> <PAR> service : medicine <PAR> <PAR> chief complaint : admitted from rehabilitation for <PAR> hypotension ( systolic blood pressure to the 70s ) and <PAR> decreased urine output <PAR> <PAR> history of present illness : the patient is a 76 - year - old <PAR> male who had been hospitalized at the [ hospital1 190 ] from [ 10 - 11 ] through [ 10 - 19 ] of [ 2101 ] <PAR> after undergoing a left femoral - at bypass graft and was <PAR> subsequently discharged to a rehabilitation facility <PAR> <PAR> on [ 2101/10/20 ] , he presented again to the [ hospital1 346 ] after being found to have a systolic <PAR> blood pressure in the 70s and no urine output for 17 hours <PAR> a foley catheter placed at the rehabilitation facility <PAR> yielded 100 cc of murky / brown urine there may also have <PAR> been purulent discharge at the penile meatus at this time <PAR> <PAR> on presentatio

The text field has now been fully cleaned and tokenised. We can proceed to extract the first few tokens to use as a hint and move forth with joining other tables and partitioning the dataset.

In [25]:
counter = 0
def produce_hint(text):
    global counter
    l = text.split()
    counter += 1
    if (counter % 10000) == 0:
        print (counter)
    return ' '.join(l[:10]) # first 10 tokens

df['HINT'] = df['TEXT'].map(lambda x: produce_hint(x))

print(df.shape)
df.head()

10000
20000
30000
40000
50000
(55404, 10)


,SUBJECT_ID,DOB,GENDER,HADM_ID,CATEGORY,CHARTDATE,ROW_ID,AGE_AT_NOTEEVENT,TEXT,HINT
0,3,2025-04-11 00:00:00,M,145834.0,Discharge summary,2101-10-31,44005,77,admission date : [ 2101/10/20 ] discharge date...,admission date : [ 2101/10/20 ] discharge date...
1,4,2143-05-12 00:00:00,F,185777.0,Discharge summary,2191-03-23,4788,48,admission date : [ 2191/3/16 ] discharge date ...,admission date : [ 2191/3/16 ] discharge date : [
2,6,2109-06-21 00:00:00,F,107064.0,Discharge summary,2175-06-15,20825,66,admission date : [ 2175/5/30 ] discharge date ...,admission date : [ 2175/5/30 ] discharge date : [
3,9,2108-01-26 00:00:00,M,150750.0,Discharge summary,2149-11-14,57115,42,"name : [ known lastname 10050 ] , [ known firs...","name : [ known lastname 10050 ] , [ known"
4,9,2108-01-26 00:00:00,M,150750.0,Discharge summary,2149-11-13,20070,42,admission date : [ 2149/11/9 ] discharge date ...,admission date : [ 2149/11/9 ] discharge date : [


Now we load all the context data from other tables. We'll be using this to construct the structured input to our encoder

In [46]:
lab_table = pd.read_sql_query('''
  SELECT "SUBJECT_ID", "CHARTTIME", "VALUE", "VALUEUOM", "FLAG","ITEMID"
  FROM "LABEVENTS"
''', cnx)

desc_lab_table = pd.read_sql_query('''
  SELECT "LABEL", "ITEMID"
  FROM "D_LABITEMS"
''', cnx)

df_labitems= pd.merge(lab_table, desc_lab_table, on=['ITEMID'])
df_labitems = df_labitems.sort_values(by=['SUBJECT_ID'])
df_labitems

,SUBJECT_ID,CHARTTIME,VALUE,VALUEUOM,FLAG,ITEMID,LABEL
0,3,2101-10-12 16:07:00,7.39,units,None,50820,pH
72,3,2101-10-15 03:30:00,29.5,pg,None,51248,MCH
71,3,2101-10-14 03:00:00,29.4,pg,None,51248,MCH
70,3,2101-10-14 03:00:00,NORMAL,None,None,51246,Macrocytes
69,3,2101-10-14 03:00:00,13.3,%,abnormal,51244,Lymphocytes
...,...,...,...,...,...,...,...
28,3,2101-10-13 15:47:00,1.5,mg/dL,abnormal,50912,Creatinine
27,3,2101-10-13 03:00:00,1.7,mg/dL,abnormal,50912,Creatinine
26,3,2101-10-15 03:30:00,105,mEq/L,None,50902,Chloride
36,3,2101-10-13 15:47:00,2.3,mg/dL,None,50960,Magnesium


In [29]:
df_prescriptions = pd.read_sql_query('''
  SELECT "SUBJECT_ID", "STARTDATE", "ENDDATE", LOWER("DRUG"), "PROD_STRENGTH"
  FROM "PRESCRIPTIONS"
  ORDER BY "SUBJECT_ID"
  --LIMIT 10000;
''', cnx)

print(df_prescriptions.shape)
df_prescriptions.head()

(4156450, 5)


,SUBJECT_ID,STARTDATE,ENDDATE,lower,PROD_STRENGTH
0,2,2138-07-18 00:00:00,2138-07-21 00:00:00,ampicillin sodium,500mg Vial
1,2,2138-07-18 00:00:00,2138-07-21 00:00:00,send 500mg vial,Send 500mg Vial
2,2,2138-07-18 00:00:00,2138-07-20 00:00:00,neo*iv*gentamicin,10mg/mL-2mL
3,2,2138-07-18 00:00:00,2138-07-20 00:00:00,syringe (neonatal) *d5w*,1 Syringe
4,4,2191-03-16 00:00:00,2191-03-23 00:00:00,heparin,5000U/ML VIAL


In [49]:
diag_table = pd.read_sql_query('''
  SELECT "SUBJECT_ID", "HADM_ID", "SEQ_NUM", "ICD9_CODE"
  FROM "DIAGNOSES_ICD" 
  -- LIMIT 100
''', cnx)

desc_diag_table = pd.read_sql_query('''
  SELECT "ICD9_CODE", "SHORT_TITLE", LOWER("LONG_TITLE")
  FROM "D_ICD_DIAGNOSES" 
''', cnx)

diag_desc = pd.merge(diag_table, desc_diag_table, on=['ICD9_CODE'])
df_diagnoses = diag_desc.sort_values(by=["SUBJECT_ID", "SEQ_NUM"])
df_diagnoses

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,SHORT_TITLE,lower
0,109,172335,1.0,40301,Mal hyp kid w cr kid V,"hypertensive chronic kidney disease, malignant..."
1,109,173633,1.0,40301,Mal hyp kid w cr kid V,"hypertensive chronic kidney disease, malignant..."
2,109,172335,2.0,486,"Pneumonia, organism NOS","pneumonia, organism unspecified"
23,109,173633,2.0,5856,End stage renal disease,end stage renal disease
4,109,172335,3.0,58281,Chr nephritis in oth dis,chronic glomerulonephritis in diseases classif...
...,...,...,...,...,...,...
69,118,147035,2.0,V053,Need prphyl vc vrl hepat,need for prophylactic vaccination and inoculat...
93,119,135738,1.0,88122,Opn wound wrist w tendon,"open wound of wrist, with tendon involvement"
94,119,135738,2.0,9032,Injury radial vessels,injury to radial blood vessels
95,119,135738,3.0,9033,Injury ulnar vessels,injury to ulnar blood vessels


In [31]:
# procedures data

hadm_IDS = list(pd.read_csv('../data/hadm_ids.csv', sep=',')['HADM_ID'])

procedure_ICD_DF = pd.read_sql_query('''
  SELECT "SUBJECT_ID", "HADM_ID", "SEQ_NUM", "ICD9_CODE"
  FROM "PROCEDURES_ICD"
  -- LIMIT 100
''', cnx)

D_procedure_ICD_DF = pd.read_sql_query('''
  SELECT "ICD9_CODE", "SHORT_TITLE", "LONG_TITLE"
  FROM "D_ICD_PROCEDURES"
''', cnx)

procedure_DF_dup= pd.merge(procedure_ICD_DF, D_procedure_ICD_DF, on=['ICD9_CODE'])
procedure_DF_dup = procedure_DF_dup.sort_values(by=['SUBJECT_ID', 'SEQ_NUM'])
df_procedures = procedure_DF_dup[procedure_DF_dup['HADM_ID'].isin(hadm_IDS)]

print(df_procedures.shape)
df_procedures.head(20)

(246178, 6)
(219744, 6)


,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,SHORT_TITLE,lower
1,3,145834,6,966,Entral infus nutrit sub,enteral infusion of concentrated nutritional s...
2,3,145834,5,3893,Venous cath NEC,"venous catheterization, not elsewhere classified"
3,3,145834,4,9672,Cont inv mec ven 96+ hrs,continuous invasive mechanical ventilation for...
4,3,145834,3,8964,Pulmon art wedge monitor,pulmonary artery wedge monitoring
5,3,145834,2,9962,Heart countershock NEC,other electric countershock of heart
6,3,145834,1,9604,Insert endotracheal tube,insertion of endotracheal tube
7,4,185777,3,3323,Other bronchoscopy,other bronchoscopy
8,4,185777,2,8872,Dx ultrasound-heart,diagnostic ultrasound of heart
9,4,185777,1,3893,Venous cath NEC,"venous catheterization, not elsewhere classified"
11,6,107064,5,9904,Packed cell transfusion,transfusion of packed cells


In [32]:
# microbiology data

df_microbiology = pd.read_sql_query('''
  SELECT 
      "SUBJECT_ID", 
      "HADM_ID", 
      MAX("CHARTDATE") AS "CHARTDATE", 
      LOWER("SPEC_TYPE_DESC"), 
      string_agg(DISTINCT(LOWER("ORG_NAME")), ', ') AS ORGANISM
  
  FROM "MICROBIOLOGYEVENTS"
  GROUP BY "SUBJECT_ID", "HADM_ID", "CHARTTIME", "SPEC_TYPE_DESC"
  ORDER BY "SUBJECT_ID", "CHARTDATE" DESC
  --LIMIT 10000;
''', cnx)

print(df_microbiology.shape)
df_microbiology.head(20)

(340305, 5)


,SUBJECT_ID,HADM_ID,CHARTDATE,lower,organism
0,2,163353,2138-07-17 00:00:00,blood culture - neonate,None
1,3,145834,2101-10-28 00:00:00,stool,None
2,3,145834,2101-10-26 00:00:00,urine,yeast
3,3,145834,2101-10-21 00:00:00,sputum,yeast
4,3,145834,2101-10-21 00:00:00,urine,yeast
5,3,145834,2101-10-21 00:00:00,catheter tip-iv,None
6,3,145834,2101-10-21 00:00:00,blood culture,None
7,3,145834,2101-10-21 00:00:00,blood culture,None
8,3,145834,2101-10-21 00:00:00,blood culture,None
9,3,145834,2101-10-21 00:00:00,blood culture ( myco/f lytic bottle),None


In [33]:
# # phenotype classification subject - we need to make sure these are part of our test set

# annotations = pd.read_csv('../data/annotations.csv')
# annotations.columns.values[0] = 'HADM_ID'
# annotations.columns.values[1] = 'SUBJECT_ID'

# pheno_subjects = list(annotations['SUBJECT_ID'])
# print(annotations.shape)
# annotations.head()

(1610, 18)


,HADM_ID,SUBJECT_ID,chart.time,cohort,Obesity,Non.Adherence,Developmental.Delay.Retardation,Advanced.Heart.Disease,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,118003,3644,118003,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,177830,97736,999999,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,185673,27694,999999,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,131938,16275,131938,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,198999,4059,198999,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0


We now split the dataset into training, validation and test sets. As we do so, we query the relevant context data and append it to the input

In [34]:
#%%timeit -n 3 -r 3

# Split the dataset in a grouped and stratified manner

def StratifiedGroupShuffleSplit(df_main):
    
    np.random.seed(768) # seeded for reproducibility
    df_main = df_main.reindex(np.random.permutation(df_main.index)) # shuffle dataset
    
    # create empty train, val and test datasets
    df_train = pd.DataFrame()
    df_val = pd.DataFrame()
    df_test = pd.DataFrame()

    hparam_mse_wgt = 0.8 # must be between 0 and 1
    assert(0 <= hparam_mse_wgt <= 1)
    train_proportion = 0.8 # must be between 0 and 1
    assert(0 <= train_proportion <= 1)
    val_test_proportion = (1-train_proportion)/2

    subject_grouped_df_main = df_main.groupby(['SUBJECT_ID'], sort=False, as_index=False)
    readmit_grouped_df_main = df_main.groupby('30_D_UNPLAN_READMIT').count()[['SUBJECT_ID']]/len(df_main)*100 
    
    # function to calculate loss
    def calc_mse_loss(df):
        grouped_df = df.groupby('30_D_UNPLAN_READMIT').count()[['SUBJECT_ID']]/len(df)*100
        df_temp = readmit_grouped_df_main.join(grouped_df, on = '30_D_UNPLAN_READMIT', how = 'left', lsuffix = '_main')
        df_temp.fillna(0, inplace=True)
        df_temp['DIFF'] = (df_temp['SUBJECT_ID_main'] - df_temp['SUBJECT_ID'])**2
        mse_loss = np.mean(df_temp['DIFF'])
        return mse_loss
    
    directory = "../data/preprocessed/"
    
    f_train = open(directory + "src-train.txt","w+")
    f_val = open(directory + "src-val.txt","w+")
    f_test = open(directory + "src-test.txt","w+")
    
    len_train = 0
    len_val = 0
    len_test = 0
    total_records = 0
    i = 0

    # loop the groups of subjects one by one
    for _, group in subject_grouped_df_main:

        total_records = len_train + len_val + len_test
        g = pd.DataFrame(group)
        subject_id = g['SUBJECT_ID'].iloc[0]
        
        pre_left = df_prescriptions['SUBJECT_ID'].searchsorted(subject_id, 'left')
        pre_right = df_prescriptions['SUBJECT_ID'].searchsorted(subject_id, 'right')
        
        lab_left = df_labitems['SUBJECT_ID'].searchsorted(subject_id, 'left')
        lab_right = df_labitems['SUBJECT_ID'].searchsorted(subject_id, 'right')
        
        diag_left = df_diagnoses['SUBJECT_ID'].searchsorted(subject_id, 'left')
        diag_right = df_diagnoses['SUBJECT_ID'].searchsorted(subject_id, 'right')
        
        proc_left = df_procedures['SUBJECT_ID'].searchsorted(subject_id, 'left')
        proc_right = df_procedures['SUBJECT_ID'].searchsorted(subject_id, 'right')
        
        micro_left = df_microbiology['SUBJECT_ID'].searchsorted(subject_id, 'left')
        micro_right = df_microbiology['SUBJECT_ID'].searchsorted(subject_id, 'right')
        
        g_prescriptions = df_prescriptions[pre_left:pre_right]
        g_labitems = df_labitems[lab_left:lab_right]
        g_diagnoses = df_diagnoses[diag_left:diag_right]
        g_procedures = df_procedures[proc_left:proc_right]
        g_microbiology = df_microbiology[micro_left:micro_right]
        
        train = False
        val = False
        test = False
        i += 1
        
        # all subjects in the phenotyping dataset need to be in the test set
        if subject_id in pheno_subjects:
            df_test = df_test.append(g, ignore_index=True)
            len_test += len(g)
            test = True
            
        # just to add something to each group to start off with - otherwise we end up dividing by 0
        elif (len_train == 0 or len_val == 0 or len_test == 0):
            if (len_train == 0):
                df_train = df_train.append(g, ignore_index=True)
                len_train += len(g)
                train = True
            elif (len_val == 0):
                df_val = df_val.append(g, ignore_index=True)
                len_val += len(g)
                val = True
            else:
                df_test = df_test.append(g, ignore_index=True)
                len_test += len(g)
                test = True
        
        # every 3rd group, balance the groups jointly by proportion and by ratio of unplanned readmissions
        elif (i % 3 == 0):
            
            mse_loss_diff_train = calc_mse_loss(df_train) - calc_mse_loss(df_train.append(g, ignore_index=True))
            mse_loss_diff_val = calc_mse_loss(df_val) - calc_mse_loss(df_val.append(g, ignore_index=True))
            mse_loss_diff_test = calc_mse_loss(df_test) - calc_mse_loss(df_test.append(g, ignore_index=True))

            len_diff_train = (train_proportion - (len_train/total_records)) * 100
            len_diff_val = (val_test_proportion - (len_val/total_records)) * 100
            len_diff_test = (val_test_proportion - (len_test/total_records)) * 100

            len_loss_diff_train = len_diff_train * abs(len_diff_train)
            len_loss_diff_val = len_diff_val * abs(len_diff_val)
            len_loss_diff_test = len_diff_test * abs(len_diff_test)

            loss_train = (hparam_mse_wgt * mse_loss_diff_train) + ((1-hparam_mse_wgt) * len_loss_diff_train)
            loss_val = (hparam_mse_wgt * mse_loss_diff_val) + ((1-hparam_mse_wgt) * len_loss_diff_val)
            loss_test = (hparam_mse_wgt * mse_loss_diff_test) + ((1-hparam_mse_wgt) * len_loss_diff_test)

            if (max(loss_train,loss_val,loss_test) == loss_train):
                df_train = df_train.append(g, ignore_index=True)
                len_train += len(g)
                train = True
            elif (max(loss_train,loss_val,loss_test) == loss_val):
                df_val = df_val.append(g, ignore_index=True)
                len_val += len(g)
                val = True
            else:
                df_test = df_test.append(g, ignore_index=True)
                len_test += len(g)
                test = True
            
            if i % 100 == 0:
                print ("Group " + str(i) + ": loss_train: " + str(loss_train) + " | " + 
                                             "loss_val: " + str(loss_val) + " | " + 
                                             "loss_test: " + str(loss_test))

        # all the other groups - divided simply by ratios of the dataset splits
        else:
            train_diff = train_proportion - (len_train/total_records)
            val_diff = val_test_proportion - (len_val/total_records)
            test_diff = val_test_proportion - (len_test/total_records)
            
            if (max(train_diff, val_diff, test_diff) == train_diff):
                df_train = df_train.append(g, ignore_index=True)
                len_train += len(g)
                train = True
            elif (max(train_diff, val_diff, test_diff) == val_diff):
                df_val = df_val.append(g, ignore_index=True)
                len_val += len(g)
                val = True
            else:
                df_test = df_test.append(g, ignore_index=True)
                len_test += len(g)
                test = True
        
        # loop through every row in the group to get relevant prescriptions and lab items before appending to file
        for j, row in enumerate(g.itertuples()):
            
            hadm_id = row[4]
            # context is all day the day before the discharge day and the day of the discharge
            chartdate = datetime.combine(parser.parse(row[6]), datetime.min.time()) + timedelta(days=1)
            cutoff = chartdate - timedelta(days=2)
            micro_cutoff = chartdate - timedelta(days=4) # greater context window for microbiology results
            
            ###### LAB RESULTS
            g_labitems.CHARTTIME = g_labitems.CHARTTIME.apply(lambda x: parser.parse(x) if type(x)==type("str") else x)
            lab_condition = np.logical_and((g_labitems.CHARTTIME >= cutoff),
                                           (g_labitems.CHARTTIME < chartdate))
            lab_items = g_labitems[lab_condition]
            lab_items = lab_items.sort_values(by=['CHARTTIME'], ascending=False)
            
            ###### PRESCRIPTIONS
            g_prescriptions.STARTDATE = g_prescriptions.STARTDATE.apply(lambda x: parser.parse(x) if type(x)==type("str") else x)
            pre_condition = np.logical_and((g_prescriptions.STARTDATE >= cutoff),
                                           (g_prescriptions.STARTDATE < chartdate))
            prescriptions = g_prescriptions[pre_condition]
            prescriptions = prescriptions.sort_values(by=['STARTDATE'], ascending=False)
            
            ###### DIAGNOSES
            diagnoses = g_diagnoses[g_diagnoses.HADM_ID == hadm_id]
            diagnoses = diagnoses.sort_values(by=['SEQ_NUM'], ascending=True)
            
            ###### PROCEDURES
            if j == 0:
                procedures = g_procedures[g_procedures.HADM_ID == hadm_id]
                procedures = procedures.sort_values(by=['SEQ_NUM'], ascending=False)
            
            ###### MICROBIOLOGY TESTS
            g_microbiology.CHARTDATE = g_microbiology.CHARTDATE.apply(lambda x: parser.parse(x) if type(x)==type("str") else x)
            micro_condition = np.logical_and((g_microbiology.CHARTDATE >= micro_cutoff),
                                             (g_microbiology.CHARTDATE < chartdate))
            microbiology = g_microbiology[micro_condition]
            microbiology = microbiology.sort_values(by=['CHARTDATE'], ascending=False)
            
            ##########################################
            lab_items_list = ""
            lab_items_length = len(lab_items)
            if (lab_items_length > 0):
                for k, lab_row in enumerate(lab_items.itertuples()):
                    flag = ""
                    if (pd.isna(lab_row[5]) == False):
                        flag = " , " + str(lab_row[5])

                    lab_items_list += str(lab_row[6]) + " , " + str(lab_row[3]) + " , " + str(lab_row[4]) + flag
                    if (k != (lab_items_length - 1)):
                        lab_items_list += " | "
            
            ##########################################
            prescriptions_list = ""
            prescriptions_length = len(prescriptions)
            if (prescriptions_length > 0):
                for k, pre_row in enumerate(prescriptions.itertuples()):
                    prescriptions_list += str(pre_row[4]) + " , " + str(pre_row[5])
                    if (k != (prescriptions_length - 1)):
                        prescriptions_list += " | "

            ##########################################
            diagnoses_list = ""
            diagnoses_length = len(diagnoses)
            if (diagnoses_length > 0):
                for k, diag_row in enumerate(diagnoses.itertuples()):
                    diagnoses_list += str(diag_row[6])
                    if (k != (diagnoses_length - 1)):
                        diagnoses_list += " | "

            ##########################################
            procedures_list = ""
            if j == 0: # only allow procedures for the most recent hospital admission for a given subject
                procedures_length = len(procedures)
                if (procedures_length > 0):
                    for k, proc_row in enumerate(procedures.itertuples()):
                        procedures_list += str(proc_row[6])
                        if (k != (procedures_length - 1)):
                            procedures_list += " | "

            ##########################################
            microbiology_list = ""
            microbiology_length = len(microbiology)
            if (microbiology_length > 0):
                for k, micro_row in enumerate(microbiology.itertuples()):
                    microbiology_list += str(micro_row[4]) + " : " + str(micro_row[5])
                    if (k != (microbiology_length - 1)):
                        microbiology_list += " | "
                        
            ##########################################
            
            if (train == True):
                f_train.write(str(row[10]) + " <H> " + str(row[3]) + " <G> " + str(row[8]) + " <A> " + 
                    str(row[11]) + " <E> " + diagnoses_list + " <D> " + procedures_list + " <P> " +
                    prescriptions_list + " <M> " + microbiology_list + " <T> " + lab_items_list + " <L>" + "\n")
            elif (val == True):
                f_val.write(str(row[10]) + " <H> " + str(row[3]) + " <G> " + str(row[8]) + " <A> " + 
                    str(row[11]) + " <E> " + diagnoses_list + " <D> " + procedures_list + " <P> " +
                    prescriptions_list + " <M> " + microbiology_list + " <T> " + lab_items_list + " <L>" + "\n")
            else:
                f_test.write(str(row[10]) + " <H> " + str(row[3]) + " <G> " + str(row[8]) + " <A> " + 
                    str(row[11]) + " <E> " + diagnoses_list + " <D> " + procedures_list + " <P> " +
                    prescriptions_list + " <M> " + microbiology_list + " <T> " + lab_items_list + " <L>" + "\n")

    f_train.close()
    f_val.close()
    f_test.close()
    
    return df_train, df_val, df_test

In [35]:
src_train, src_val, src_test = StratifiedGroupShuffleSplit(df)

Group 300: loss_train: 5.069012296392446 | loss_val: 4.8227817205867565 | loss_test: -19.780522923056267
Group 600: loss_train: 1.316655124653739 | loss_val: 1.1218836565096928 | loss_test: -4.8692867036011105
Group 900: loss_train: 1.9920830011936124 | loss_val: 1.1525150329655116 | loss_test: -6.175047245606615
Group 1200: loss_train: 1.6574239953381735 | loss_val: 1.6693122345605977 | loss_test: -6.653451218155096
Group 1800: loss_train: 1.4132664984663943 | loss_val: 1.371357749929187 | loss_test: -5.568933114442709
Group 2100: loss_train: 1.6629966710645685 | loss_val: 1.4804327037832143 | loss_test: -6.2815528001621015
Group 2400: loss_train: 1.633821328136122 | loss_val: 1.5590384180621404 | loss_test: -6.384843592531771
Group 2700: loss_train: 1.2870772833783801 | loss_val: 1.292007035458223 | loss_test: -5.158163926218518
Group 3000: loss_train: 1.3511716861342273 | loss_val: 1.1834230913779005 | loss_test: -5.063632360299104
Group 3300: loss_train: 1.2171199485216275 | loss_v

Group 24300: loss_train: 7.75535783808452e-07 | loss_val: -2.3750783379019067e-06 | loss_test: 4.3623887838815507e-07
Group 24600: loss_train: 2.329899056583208e-06 | loss_val: -1.7117625721817254e-06 | loss_test: -4.7548960338742146e-08
Group 24900: loss_train: 4.2044534033443346e-07 | loss_val: -7.474583828096163e-07 | loss_test: 4.671614892506444e-08
Group 25200: loss_train: 1.1471310088643923e-06 | loss_val: -2.8678275221543336e-07 | loss_test: -2.8678275221543336e-07
Group 25500: loss_train: 7.216319207960911e-07 | loss_val: -1.8040798020007722e-07 | loss_test: -1.8040798020007722e-07
Group 25800: loss_train: 4.431359327236691e-08 | loss_val: -1.1078398318222377e-08 | loss_test: -1.1078398318222377e-08
Group 26100: loss_train: 7.362655820268133e-06 | loss_val: -6.97056172334672e-07 | loss_test: -3.528846872437282e-06
Group 26400: loss_train: 4.283917038691543e-08 | loss_val: -3.855525334845511e-07 | loss_test: 1.713566815471479e-07
Group 26700: loss_train: 4.2112971494038775e-06 |

Now that we have finally split the dataset into training, evaluation and test datasets, let's just inspect the stratification to ensure that we have successfully managed to have a uniform ratio of unplanned readmissions across all three datasets. And that they have been split correctly in the ratio 8:1:1

In [36]:
print(len(src_train), len(src_val), len(src_test))

readmit_grouped_df_main = df.groupby('30_D_UNPLAN_READMIT').count()[['SUBJECT_ID']]/len(df)*100
grouped_train = src_train.groupby('30_D_UNPLAN_READMIT').count()[['SUBJECT_ID']]/len(src_train)*100
grouped_val = src_val.groupby('30_D_UNPLAN_READMIT').count()[['SUBJECT_ID']]/len(src_val)*100
grouped_test = src_test.groupby('30_D_UNPLAN_READMIT').count()[['SUBJECT_ID']]/len(src_test)*100

df_summary = readmit_grouped_df_main.join(grouped_train, on = '30_D_UNPLAN_READMIT', how = 'left', rsuffix = '_train')
df_summary = df_summary.join(grouped_val, on = '30_D_UNPLAN_READMIT', how = 'left', rsuffix = '_val')
df_summary = df_summary.join(grouped_test, on = '30_D_UNPLAN_READMIT', how = 'left', rsuffix = '_test')
df_summary.fillna(0, inplace=True)
df_summary

44323 5541 5540


,SUBJECT_ID,SUBJECT_ID_train,SUBJECT_ID_val,SUBJECT_ID_test
30_D_UNPLAN_READMIT,,,,
Y,100.0,100.0,100.0,100.0


The ratio of 8:1:1 is roughly correct. The ratio of unplanned readmissions is also not too bad. However, the test dataset seems to contain considerably more unplanned readmissions: 13% vs 6%. 

In [39]:
print("# of Discharge summaries for subjects in phenotyping dataset: ", len(src_test[src_test['SUBJECT_ID'].isin(pheno_subjects)]))
print("# of records in the phenotyping dataset",len(pheno_subjects))
print("# of unique subjects in the phenotyping dataset",len(annotations.SUBJECT_ID.unique()))

# of Discharge summaries for subjects in phenotyping dataset:  3672
# of records in the phenotyping dataset 1610
# of unique subjects in the phenotyping dataset 1045


On closer inspection, this appears to be due to the fact that we require all subjects in the phenotyping dataset to be in the test set. Despite comprising only 1045 unique subjects, these subjects are responsible for 3672 discharge summaries - much higher than average. This naturally also corresponds to a much greater incidence of 30d unplanned readmission, hence the greater incidence for this in the test dataset. The difference between 13% and 6% is small enough that we will tolerate it.

We have saved our source datasets, now let's now save our target and reference datasets: 

In [40]:
# create target files

tgt_train = pd.DataFrame(src_train, columns = ["TEXT"])
tgt_val = pd.DataFrame(src_val, columns = ["TEXT"])
tgt_test = pd.DataFrame(src_test, columns = ["TEXT"])

print(tgt_test.shape)
tgt_test.head()

(5540, 1)


,TEXT
0,admission date : [ 2164/8/31 ] discharge date ...
1,admission date : [ 2135/4/21 ] discharge date ...
2,"name : [ known lastname ] , [ known firstname ..."
3,admission date : [ 2133/11/27 ] discharge date...
4,admission date : [ 2139/7/22 ] discharge date ...


In [41]:
# save target files to disk

directory = "../data/preprocessed/"

np.savetxt(directory + 'tgt-train.txt', tgt_train, fmt='%s', newline=os.linesep)
np.savetxt(directory + 'tgt-val.txt', tgt_val, fmt='%s', newline=os.linesep)
np.savetxt(directory + 'tgt-test.txt', tgt_test, fmt='%s', newline=os.linesep)

In [42]:
# save reference files to disk

src_train.to_csv(directory + 'ref_train.tsv', sep='\t', index = False)
src_val.to_csv(directory + 'ref_val.tsv', sep='\t', index = False)
src_test.to_csv(directory + 'ref_test.tsv', sep='\t', index = False)